In [21]:
import pandas as pd

df = pd.read_csv('data/train.csv')
df = df.drop(columns=['Id'])

In [22]:
df_num = df.select_dtypes(include = ['float64', 'int64'])
df = df_num[['LotArea','MasVnrArea','TotalBsmtSF','GrLivArea','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','SalePrice']]
df = df.dropna()
df.head()

,LotArea,MasVnrArea,TotalBsmtSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,SalePrice
0,8450,196.0,856,1710,548,0,61,0,1,0,2,1,3,1,8,208500
1,9600,0.0,1262,1262,460,298,0,0,0,1,2,0,3,1,6,181500
2,11250,162.0,920,1786,608,0,42,0,1,0,2,1,3,1,6,223500
3,9550,0.0,756,1717,642,0,35,272,1,0,1,0,3,1,7,140000
4,14260,350.0,1145,2198,836,192,84,0,1,0,2,1,4,1,9,250000


In [23]:
# split data set into train and validation
train = df.sample(frac=0.8, random_state=200)
validation = df.drop(train.index)
print(train.shape)
print(validation.shape)

(1162, 16)
(290, 16)


In [39]:
# split the columns into features and target; ie x y split

target = 'SalePrice'
train_x = train.drop(target, axis=1)
train_y = train[target]

validation_x = validation.drop(target, axis=1)
validation_y = validation[target]

train_x.head()

,LotArea,MasVnrArea,TotalBsmtSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd
1435,8400,237.0,1319,1537,462,0,36,0,1,0,1,1,3,1,7
207,12493,0.0,1100,1100,312,355,0,0,1,0,1,0,3,1,6
1346,20781,0.0,1568,2156,508,0,80,0,0,0,2,0,3,1,9
540,14601,584.0,1838,1838,765,270,68,0,1,0,2,0,2,1,8
433,10839,0.0,926,1604,470,0,36,0,0,0,2,1,3,1,7


In [35]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# create the model
rfr_model = RandomForestRegressor(n_estimators=100, max_depth=10)

# fit the model
rfr_model.fit(train_x, train_y)

# make predictions
validation_predictions = rfr_model.predict(validation_x)

# print the mean squared error
print(mean_squared_error(validation_y, validation_predictions))

1479894336.1344929


In [36]:
from sklearn.metrics import r2_score

print(r2_score(validation_y, validation_predictions))

0.7924750155993103


In [37]:
from sklearn.ensemble import GradientBoostingRegressor

# create the model
gbr_model = GradientBoostingRegressor(loss='squared_error')

gbr_model.fit(train_x, train_y)

print(gbr_model.score(validation_x, validation_y))

0.8062487981346209
